# Surrogate Model Client Tools
In this notebook, we set up the client used for controlling and displaying the variables we served in the SurrogateModelServer notebook. The input and output variables built during model development are passed to `lume-epics` widgets. The `lume-epics` `Controller` object facilitates get/puts over EPICS. In this demo, we configure the controller to use pvAccess.

## Load the model variables
`lume-model` defines utility functions for saving models during model development and subsequent loading.

In [ ]:
from lume_model.utils import load_variables

prefix = "test" # Prefix used by our server
variable_filename =  "files/surrogate_model_variables_2.pickle"
input_variables, output_variables = load_variables(variable_filename)

## Create Controller

The controller is initialized with a protocol indicator for reading process variables (`pva` for pvAccess and `ca` for Channel Access). The controller updates both the Channel Access and pvAccess variables on puts; however, it can be configured to operate using a single protocol:

```
controller = Controller("pva", set_ca=False)
```

In [ ]:
from lume_epics.client.controller import Controller
controller = Controller("pva")

##  Build sliders
The `lume-epics` build_slider utility will render bokeh `Slider` objects for each `lume-model` variable passed.

In [ ]:
from lume_epics.client.widgets.controls import build_sliders

inputs = [
          input_variables["distgen:r_dist:sigma_xy:value"], 
          input_variables["distgen:t_dist:length:value"],
          input_variables["SOL1:solenoid_field_scale"],
          input_variables["CQ01:b1_gradient"],
          input_variables["SQ01:b1_gradient"],
          input_variables["L0A_phase:dtheta0_deg"],
          input_variables["end_mean_z"]
        ]

sliders = build_sliders(inputs, controller, prefix)

## Build image plot
The `ImagePlot` object accepts a list of `lume-model` `ImageVariables`.

In [ ]:
from lume_epics.client.widgets.plots import ImagePlot
from bokeh import palettes

# create image plot
output_variables = [output_variables["x:y"]]
image_plot = ImagePlot(output_variables, controller, prefix)


# build the plot using specific bokeh palette
pal = palettes.viridis(256)
image_plot.build_plot(palette=pal)

# Use bokeh to render the application

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.resources import INLINE

# load bokeh
output_notebook(INLINE) 

# function for rendering the application in the bokeh server
def render_app(doc):
    doc.title = "Demo App"
    doc.add_root(
        column(
            row(column(sliders, width=350), column(image_plot.plot, width=350)), 
        )
    )
    doc.add_periodic_callback(image_plot.update, 250) # add callback to update the image 

    
show(render_app)